## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

In [3]:
import numpy as np
import pandas as pd
import numba
from numba import jit, njit

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
A = np.random.randint(0, 1000, size=1000000)
B = A+100
B.mean()

599.576798

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [3]:
df = pd.DataFrame(np.random.randint(0, 1000, size=(2000000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2000000, replace=True)
def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)
g(df).head()

,col1,col2,col3,col4,key
13,509,588,933,455,a
19,755,773,514,345,a
23,92,514,52,621,a
31,999,196,625,752,a
45,458,931,142,569,a


## Лабораторная работа 3

In [5]:
 pip install line_profiler


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30183 sha256=96e0ea841e9f0f9cb7d9478ad9b13b6fa6fcbce1b8023595ebcb3ad5fce14461
  Stored in directory: c:\users\lisa\appdata\local\pip\cache\wheels\6a\37\3e\d9e8ebaf73956a3ebd2ee41869444dbd2a702d7142bcf93c42
Successfully built memory-profiler
Note: you may need to restart the kernel to use updated packages.


In [4]:
%load_ext line_profiler

In [5]:
%load_ext memory_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [221]:
recipes = pd.read_csv("recipes_sample.csv", sep=",", parse_dates=['submitted'])
reviews = pd.read_csv("reviews_sample.csv", sep=",", parse_dates=['date'], index_col=0)
reviews.reset_index(drop = True, inplace=True)

In [12]:
def mean_A(reviews):
    ratS = 0
    k = 0
    for i, r in reviews.iterrows():
        if r.date.year == 2010:
            ratS += r['rating']
            k += 1
    return ratS/k

In [13]:
def mean_B(reviews):
    rv = reviews[reviews.date.dt.year == 2010]
    ratS = 0
    for i, r in rv.iterrows():
        ratS += r['rating']
    return ratS/rv.shape[0]

In [14]:
def mean_C(reviews):
    return reviews[reviews.date.dt.year == 2010]['rating'].mean()

In [1]:
%%time
print(mean_A(reviews))

` not found.


In [16]:
%%time
mean_B(reviews)

Wall time: 583 ms


4.4544402182900615

In [17]:
%%time
print(mean_C(reviews))

4.4544402182900615
Wall time: 10.1 ms


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [18]:
%lprun -f mean_A mean_A(reviews)

Timer unit: 1e-07 s

Total time: 15.679 s
File: <ipython-input-12-fb9491a545bc>
Function: mean_A at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean_A(reviews):
     2         1         13.0     13.0      0.0      ratS = 0
     3         1          5.0      5.0      0.0      k = 0
     4    126697  138071513.0   1089.8     88.1      for i, r in reviews.iterrows():
     5    126696   17766043.0    140.2     11.3          if r.date.year == 2010:
     6     12094     906714.0     75.0      0.6              ratS += r['rating']
     7     12094      45276.0      3.7      0.0              k += 1
     8         1         10.0     10.0      0.0      return ratS/k

Timer unit: 1e-07 s

Total time: 33.5067 s
File: <ipython-input-131-c151a5cd4da1>
Function: mean_A at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def mean_A(reviews):
     2         1         15.0     15.0      0.0      ratS = 0
     3         1          8.0      8.0      0.0      k = 0
     4    126697  279562694.0   2206.5     83.4      for i, r in reviews.iterrows():
     5    126696   52714967.0    416.1     15.7          if r.date.year == 2010:
     6     12094    2707164.0    223.8      0.8              ratS += r['rating']
     7     12094      82230.0      6.8      0.0              k += 1
     8         1         12.0     12.0      0.0      return ratS/k

Больше всего времени тратится на интерации цикла и проверку условия

In [19]:
%lprun -f mean_B mean_B(reviews)

Timer unit: 1e-07 s

Total time: 1.36769 s
File: <ipython-input-13-521e20ed591c>
Function: mean_B at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean_B(reviews):
     2         1      96871.0  96871.0      0.7      rv = reviews[reviews.date.dt.year == 2010]
     3         1         13.0     13.0      0.0      ratS = 0
     4     12095   12609244.0   1042.5     92.2      for i, r in rv.iterrows():
     5     12094     970657.0     80.3      7.1          ratS += r['rating']
     6         1         65.0     65.0      0.0      return ratS/rv.shape[0]

Timer unit: 1e-07 s

Total time: 2.83106 s
File: <ipython-input-132-2ada19111e66>
Function: mean_B at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def mean_B(reviews):
     2         1     167908.0 167908.0      0.6      rv = reviews[reviews.date.dt.year == 2010]
     3         1         13.0     13.0      0.0      ratS = 0
     4     12095   24993617.0   2066.4     88.3      for i, r in rv.iterrows():
     5     12094    3148955.0    260.4     11.1          ratS += r['rating']
     6         1         81.0     81.0      0.0      return ratS/rv.shape[0]

Больше всего времени тратится на интерации цикла

In [20]:
%lprun -f mean_C mean_C(reviews)

Timer unit: 1e-07 s

Total time: 0.0090285 s
File: <ipython-input-14-5852f625aa0e>
Function: mean_C at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean_C(reviews):
     2         1      90285.0  90285.0    100.0      return reviews[reviews.date.dt.year == 2010]['rating'].mean()

Timer unit: 1e-07 s

Total time: 0.0171297 s
File: <ipython-input-133-35496f1ea080>
Function: mean_C at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def mean_C(reviews):
     2         1     171297.0 171297.0    100.0      return reviews[reviews.date.dt.year == 2010]['rating'].mean()

In [21]:
def mean_B1(reviews):
    rv = reviews[reviews.date.dt.year == 2010]
    return rv['rating'].sum()/rv.shape[0]

In [22]:
mean_B1(reviews)

4.4544402182900615

In [23]:
%lprun -f mean_B1 mean_B1(reviews)

Timer unit: 1e-07 s

Total time: 0.0091159 s
File: <ipython-input-21-9d4b86c33de6>
Function: mean_B1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mean_B1(reviews):
     2         1      87788.0  87788.0     96.3      rv = reviews[reviews.date.dt.year == 2010]
     3         1       3371.0   3371.0      3.7      return rv['rating'].sum()/rv.shape[0]

Timer unit: 1e-07 s

Total time: 0.0196799 s
File: <ipython-input-140-4f01dfa41a1e>
Function: mean_B1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def mean_B1(reviews):
     2         1     184861.0 184861.0     93.9      rv = reviews[reviews.date.dt.year == 2010]
     3         1      11938.0  11938.0      6.1      return rv['rating'].sum()/rv.shape[0]

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [24]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [25]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-07 s

Total time: 44.3078 s
File: <ipython-input-24-f2b5c45f390a>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count(df):
     2         1         14.0     14.0      0.0      word_reviews = {}
     3    126680  138116700.0   1090.3     31.2      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679   18725560.0    147.8      4.2          recipe_id, review = row['recipe_id'], row['review']
     5    126679    3952675.0     31.2      0.9          words = review.split(' ')
     6   6918689   16899927.0      2.4      3.8          for word in words:
     7   6792010   27352082.0      4.0      6.2              if word not in word_reviews:
     8    174426     679991.0      3.9      0.2                  word_reviews[word] = []
     9   6792010   27985110.0      4.1      6.3              word_reviews[word].append(recipe_id)
    10 

Timer unit: 1e-07 s

Total time: 94.8331 s
File: <ipython-input-37-f2b5c45f390a>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1         44.0     44.0      0.0      word_reviews = {}
     3    126680  307372721.0   2426.4     32.4      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679   65111357.0    514.0      6.9          recipe_id, review = row['recipe_id'], row['review']
     5    126679    7858419.0     62.0      0.8          words = review.split(' ')
     6   6918689   30722058.0      4.4      3.2          for word in words:
     7   6792010   46552338.0      6.9      4.9              if word not in word_reviews:
     8    174426    1295688.0      7.4      0.1                  word_reviews[word] = []
     9   6792010   50238494.0      7.4      5.3              word_reviews[word].append(recipe_id)
    10         1         24.0     24.0      0.0      word_reviews_count = {}
    11    126680  291349315.0   2299.9     30.7      for _, row in df.dropna(subset=['review']).iterrows():
    12    126679   37424069.0    295.4      3.9          review = row['review']
    13    126679    7654003.0     60.4      0.8          words = review.split(' ')
    14   6918689   31503737.0      4.6      3.3          for word in words:
    15   6792010   71249044.0     10.5      7.5              word_reviews_count[word] = len(word_reviews[word])
    16         1         18.0     18.0      0.0      return word_reviews_count

Во-первых, данную функцию как минимум замедляет наличие двух похожих циклов. Во-вторых, мы храним индексы, которые не нужны. В-третьих, сама функция работает не совсем корректно.

In [26]:
def get_word_reviews_count2(df):
    k = 0
    word_reviews = dict.fromkeys(" ".join(reviews['review'].dropna()).split(' '), 0)
    for row in df['review'].dropna():
        for word in set(row.split(' ')):
            word_reviews[word] += 1
    return word_reviews

In [27]:
%lprun -f get_word_reviews_count2 get_word_reviews_count2(reviews)

Timer unit: 1e-07 s

Total time: 5.42143 s
File: <ipython-input-26-d6356bc3e693>
Function: get_word_reviews_count2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count2(df):
     2         1         13.0     13.0      0.0      k = 0
     3         1    8729405.0 8729405.0     16.1      word_reviews = dict.fromkeys(" ".join(reviews['review'].dropna()).split(' '), 0)
     4    126680     571151.0      4.5      1.1      for row in df['review'].dropna():
     5   5513986   20597172.0      3.7     38.0          for word in set(row.split(' ')):
     6   5387307   24316570.0      4.5     44.9              word_reviews[word] += 1
     7         1         18.0     18.0      0.0      return word_reviews

Timer unit: 1e-07 s

Total time: 9.2698 s
File: <ipython-input-45-c95051c92bb4>
Function: get_word_reviews_count2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count2(df):
     2         1         21.0     21.0      0.0      k = 0
     3         1   16691021.0 16691021.0     18.0      word_reviews = dict.fromkeys(" ".join(reviews['review'].dropna()).split(' '), 0)
     4    126680    1109065.0      8.8      1.2      for row in df['review'].dropna():
     5                                           
     6   5513986   34795738.0      6.3     37.5          for word in set(row.split(' ')):
     7   5387307   40102150.0      7.4     43.3              word_reviews[word] += 1
     8         1         30.0     30.0      0.0      return word_reviews

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [400]:
rv = reviews[reviews['rating'] != 0]

In [401]:
def mean_arr(df):
    df1 = pd.merge(df, df.groupby('recipe_id')['rating'].mean(), left_on='recipe_id', right_on='recipe_id', how='left')
    return np.array(df1['rating_x']), np.array(df1['rating_y'])

In [402]:
rating, rat_mean = mean_arr(rv)

In [403]:
def mape_1(rating, rat_mean):
    s = 0
    for i in range(rating.size):
        s += abs((rating[i] - rat_mean[i]) /rating[i])
    return 100 * s/rating.size

In [404]:
mape_1(rating, rat_mean)

11.171550259058838

In [405]:
%lprun -f mape_1 mape_1(rating, rat_mean)

Timer unit: 1e-07 s

Total time: 0.377799 s
File: <ipython-input-403-f49b29e03961>
Function: mape_1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mape_1(rating, rat_mean):
     2         1         16.0     16.0      0.0      s = 0
     3    119892     373265.0      3.1      9.9      for i in range(rating.size):
     4    119891    3404671.0     28.4     90.1          s += abs((rating[i] - rat_mean[i]) /rating[i])
     5         1         37.0     37.0      0.0      return 100 * s/rating.size

In [406]:
@njit
def mape_2(rating, rat_mean):
    s = 0
    for i in range(rating.size):
        s += abs((rating[i] - rat_mean[i]) /rating[i])
    return 100 * s/rating.size

In [407]:
mape_2(rating, rat_mean)

11.171550259058838

In [408]:
%lprun -f mape_2 mape_2(rating, rat_mean)

Timer unit: 1e-07 s

Total time: 0 s
File: <ipython-input-406-8191d7a16ac1>
Function: mape_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           @njit
     2                                           def mape_2(rating, rat_mean):
     3                                               s = 0
     4                                               for i in range(rating.size):
     5                                                   s += abs((rating[i] - rat_mean[i]) /rating[i])
     6                                               return 100 * s/rating.size

In [409]:
def mape_3(rating, rat_mean):
    return 100 * np.mean(np.abs((rating - rat_mean) /rating))

In [410]:
mape_3(rating, rat_mean)

11.171550259058085

In [411]:
%lprun -f mape_3 mape_3(rating, rat_mean)

Timer unit: 1e-07 s

Total time: 0.0009301 s
File: <ipython-input-409-71b339867b9d>
Function: mape_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mape_3(rating, rat_mean):
     2         1       9301.0   9301.0    100.0      return 100 * np.mean(np.abs((rating - rat_mean) /rating))

In [412]:
@njit
def mape_4(rating, rat_mean):
    return 100 * np.mean(np.abs((rating - rat_mean) /rating))

In [413]:
mape_4(rating, rat_mean)

11.17155025905884

In [414]:
%lprun -f mape_4 mape_4(rating, rat_mean)

Timer unit: 1e-07 s

Total time: 0 s
File: <ipython-input-412-3920eccc31f9>
Function: mape_4 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           @njit
     2                                           def mape_4(rating, rat_mean):
     3                                               return 100 * np.mean(np.abs((rating - rat_mean) /rating))

In [415]:
#неверно трактованное условие
def mape_3_(df):
    mn = df.groupby('recipe_id')['rating'].mean()
    mape = {}
    for ind in mn.index:
        revs = df[df['recipe_id'] == ind]['rating']
        revs = abs((revs - mn[ind]) / revs)
        mape[ind] = 100 / revs.shape[0] * revs.sum()
    return pd.Series(mape)


In [416]:
%lprun -f mape_3_ mape_3_(rv)

Timer unit: 1e-07 s

Total time: 68.9809 s
File: <ipython-input-415-7a697fd356e8>
Function: mape_3_ at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def mape_3_(df):
     3         1      97799.0  97799.0      0.0      mn = df.groupby('recipe_id')['rating'].mean()
     4         1         14.0     14.0      0.0      mape = {}
     5     27441     461737.0     16.8      0.1      for ind in mn.index:
     6     27440  457289303.0  16665.1     66.3          revs = df[df['recipe_id'] == ind]['rating']
     7     27440  192231039.0   7005.5     27.9          revs = abs((revs - mn[ind]) / revs)
     8     27440   39676876.0   1446.0      5.8          mape[ind] = 100 / revs.shape[0] * revs.sum()
     9         1      52530.0  52530.0      0.0      return pd.Series(mape)